In [1]:
# # Importa biblioteca para utilização de ferramentas de sistema operacional
# import os

# # Clona o repositório, com os datasets, se não existir uma versão dele
# if not os.path.exists('/content/iia-trabalho-1'):
#   !git clone https://github.com/smmstakes/iia-trabalho-1.git

# # Instalando biblioteca para geração de dados falsos
# !pip install Faker

In [2]:
# Importa todas as bibliotecas a serem utilizadas no projeto
import time
import json
import pickle
import joblib
import unicodedata

import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors

from faker import Faker
from geopy.distance import geodesic
from geopy.geocoders import Nominatim

In [3]:
# # Lista com as cidades dos produtores
with open("./data/json/cities_list.json", "r") as file:
    cities_list = json.load(file)

# Dicionário com uma tupla com a localização de cada cidade
# locations = get_locations(cities_list)
with open("./data/json/locations.json", "r") as file:
  locations = json.load(file)

with open("./data/json/producers_locations.json", "r") as file:
  producers_location = json.load(file)

with open("./data/json/producers_ra.json", "r") as file:
    producers_ra = json.load(file)

# Lista de cada produto ofertado pelos produtores
with open("./data/json/products_list.json", "r") as file:
    products_list = json.load(file)

In [4]:
# Definindo funções auxiliares

def get_locations(cities_list: list) -> dict:
  # Definindo um agente para pegar as coordenadas
  geolocator = Nominatim(user_agent="loc_producers", timeout=10)
  locations = {}

  for city in cities_list:
    try:
      time.sleep(1)

      # Requisição para uma cidade na lista de cidade
      loc = geolocator.geocode(f"{city}, DF, Brasil")
      locations[city] = (loc.latitude, loc.longitude) if loc else (None, None)

    except Exception as e:
      print(f"Erro ao tentar achar: {city}: {e}")
      locations[city] = (None, None)

  return locations

def name_formatter(name: str) -> str:
    # Remove acentos e normaliza caracteres
    new_name = unicodedata.normalize('NFKD', name)

    # Substitui espaços por underscores e remove caracteres especiais
    return new_name.encode('ascii', 'ignore').decode('ascii').replace(' ', '_')

def producer_infos(producer: str) -> tuple:
  try:
    prod, formatted_local = producer.split("_", 1)

    if prod.startswith("Rede"):
      return "Rede Terra" ,"Santa Maria", locations["Santa Maria"][0], locations["Santa Maria"][1]

    for city in cities_list:
      if name_formatter(city) == formatted_local:
        correct_location = city
        lat, lon = locations[correct_location]
        return prod, correct_location, lat, lon

  except ValueError as e:
      return "Desconhecido", "Desconhecido", 0.0, 0.0
  
def producer_name_formatter(producers: list) -> list:
    prods_formatted = []
    for producer, locals in producers.items():
        for ra in locals:
            produtor_formatado = producer.replace(' ', '_')
            ra = name_formatter(ra)
            prods_formatted.append(f"{produtor_formatado}_{ra}")
    return prods_formatted

def get_distance(coord1, coord2):
    return geodesic(coord1, coord2).kilometers

In [5]:
# Concatenando nome do produtor com suas RAs

# producers_formatted = producer_name_formatter(producers_ra)
with open("./data/json/producers_formatted.json", "r") as file:
    producers_formatted =  json.load(file)

In [6]:
# Definindo número inicial para geração pseudo-aleatória
np.random.seed(53)

matrix_size = 2000

# Base do DataFrame de produtos
products = {
    "produto": [],
    "organico": [],
    "nome_produtor": [],
    "local": [],
    "latitude": [],
    "longitude": []
}

# Adicionando informações sobre produtos vendidos por cada produtor
for _ in range(matrix_size):
    products["produto"].append(np.random.choice(products_list))
    products["organico"].append(np.random.choice([0, 1]))

    prod = np.random.choice(producers_formatted)
    producer, local, lat, lon = producer_infos(prod)

    products["nome_produtor"].append(producer)
    products["local"].append(local)
    products["latitude"].append(lat)
    products["longitude"].append(lon)

# Gerando DataFrame e exportando para csv
df_products = pd.DataFrame(products).drop_duplicates()
df_products.to_csv('./data/datasets/producers.csv', index=False)
df_products.head(20)

,produto,organico,nome_produtor,local,latitude,longitude
0,Maracujá,1,Cooperbrasília,Sobradinho,-15.650053,-47.784845
1,Mamão,0,Coopbrasil,Planaltina,-15.618195,-47.655570
2,Cenoura,1,Aspaf,Guará,-15.823563,-47.976816
3,Graviola,0,Coopbrasil,Brazlândia,-15.680890,-48.194262
4,Cebola,0,Rede Terra,Santa Maria,-16.017123,-48.013133
5,Morango,1,Cooper-Horti,Paranoá,-15.775440,-47.779763
6,Brócolis,1,Coopbrasil,Ceilândia,-15.817339,-48.104577
7,Mandioca,0,Rede Terra,Santa Maria,-16.017123,-48.013133
8,Cebola,0,Asproc,Samambaia,-15.876999,-48.088100
9,Cenoura,0,Coopbrasil,Samambaia,-15.876999,-48.088100


In [7]:
# Gerador de dados falsos
fake = Faker()

n_reviews = 7500
n_users = 2500

# Criando usuários ficticios
users = [fake.uuid4() for _ in range(n_users)]

# Basa para o DataFrame de avaliação de produtos
reviews = {
          "id_usuario": [],
          "produto": [],
          "organico": [],
          "nome_produtor": [],
          "local": [],
          "avaliacao": []
        }

for _ in range(n_reviews):
  reviews["id_usuario"].append(np.random.choice(users))

  id_produto = np.random.choice(len(df_products))

  reviews["produto"].append(df_products.iloc[id_produto]["produto"])
  reviews["organico"].append(df_products.iloc[id_produto]["organico"])
  reviews["nome_produtor"].append(df_products.iloc[id_produto]["nome_produtor"])
  reviews["local"].append(df_products.iloc[id_produto]["local"])
  reviews["avaliacao"].append(np.random.randint(1, 6))

df_reviews = pd.DataFrame(reviews).drop_duplicates()
df_reviews.to_csv('./data/datasets/reviews.csv', index=False)
df_reviews.head(20)

,id_usuario,produto,organico,nome_produtor,local,avaliacao
0,6a1a6cb2-4672-40e8-b774-b82910b57efc,Mandioca,0,Coopbrasil,Ceilândia,1
1,be7f15dd-e5a7-49e1-ad3a-f1306a24ee92,Coco,0,Prorural,Paranoá,3
2,efd731af-89c0-4ff7-b4a8-e137f0ae6f13,Agrião,1,Aspaf,Núcleo Bandeirante,2
3,4e802481-8285-4f7f-b278-9966f51fd711,Pimentão,0,Asproc,Planaltina,3
4,df2d2d0b-c2bf-487d-b4d4-a0ea3918d824,Goiaba,0,Asphor,Gama,4
5,05cdc25e-7ebe-4f38-b746-168722d01f38,Goiaba,1,Aspaf,Núcleo Bandeirante,4
6,d8091ee9-92c1-461b-83f1-76e522e09fce,Mamão,0,Aspaf,Núcleo Bandeirante,2
7,f53e5878-709f-4091-91a7-5e5338cc5c12,Beterraba,0,Astraf,Guará,1
8,a88dcffd-fbb8-4d9e-9f09-4aa5ec7e1b31,Banana,0,Coopebraz,Recanto das Emas,1
9,eda8ec4c-79a2-4556-87bf-94c1e92039b0,Batata,0,Coopbrasil,Planaltina,4


In [8]:
utility_matrix = pd.pivot_table(
  df_reviews,
  values='avaliacao',
  index='nome_produtor',
  columns='produto',
  fill_value=0
  ).round(2)

utility_matrix.to_csv('./data/datasets/matrix_reviews.csv')
utility_matrix.head(20)

produto,Abacate,Abóbora,Agrião,Alface,Atemóia,Banana,Batata,Berinjela,Beterraba,Brócolis,...,Maracujá,Morango,Pepino,Pimentão,Pitaia,Quiabo,Repolho,Tangerina,Tomate,Uva
nome_produtor,,,,,,,,,,,,,,,,,,,,,
Afeca,0.00,2.86,3.60,2.69,2.62,0.00,0.00,4.00,2.40,2.42,...,0.00,1.33,2.00,0.00,3.71,0.00,1.00,3.45,3.60,0.00
Agrifam,2.64,2.71,2.65,3.42,2.64,2.62,3.42,2.92,3.23,3.54,...,2.90,2.79,3.27,2.67,2.63,3.39,2.88,2.83,3.64,4.00
Amista,3.50,0.00,0.00,2.40,3.40,2.54,0.00,2.50,3.40,0.00,...,2.50,0.00,3.00,2.89,2.64,2.40,3.00,3.75,2.57,4.00
AsSpag,3.00,0.00,2.78,2.57,3.27,2.40,2.50,0.00,2.83,3.92,...,4.00,0.00,3.00,3.18,2.50,3.20,0.00,3.43,2.77,3.31
Aspaf,2.63,2.58,3.40,2.92,3.10,3.00,3.31,2.73,1.60,2.54,...,3.65,2.88,2.00,3.20,2.90,2.80,2.57,3.25,2.55,2.95
Asphor,3.50,1.50,3.30,3.17,2.73,3.07,3.24,2.08,2.67,3.25,...,3.50,3.60,3.14,2.00,3.00,2.67,3.46,2.83,3.00,3.00
Asproc,3.14,3.20,2.74,3.33,3.06,3.37,5.00,2.93,3.37,3.24,...,3.08,3.05,0.00,2.54,2.96,2.40,2.50,2.60,2.79,2.86
Aspronte,4.00,3.00,3.10,3.00,3.08,3.00,2.84,2.83,2.00,3.50,...,2.40,2.92,3.11,2.82,2.92,2.50,2.93,2.80,3.11,2.40
Astraf,3.00,2.86,3.62,3.60,0.00,0.00,4.00,2.25,2.94,3.80,...,3.20,3.00,4.40,3.18,0.00,2.50,3.00,2.40,3.70,3.14


In [9]:
# Juntando os dados para formar uma base de recomendação

# 1. Juntar as avaliações com os dados dos produtores
df_full_reviews = df_reviews.merge(df_products, on=["nome_produtor", "produto", "organico", "local"], how="inner")

df_full_reviews.to_csv("./data/datasets/df_full_reviews.csv", index=False)

# Inicializar encoders
le_usuario = LabelEncoder()
le_produto = LabelEncoder()
le_produtor = LabelEncoder()
le_local = LabelEncoder()

# Codificar colunas categóricas
df_full_reviews["usuario_id"] = le_usuario.fit_transform(df_full_reviews["id_usuario"])
df_full_reviews["produto_id"] = le_produto.fit_transform(df_full_reviews["produto"])
df_full_reviews["produtor_id"] = le_produtor.fit_transform(df_full_reviews["nome_produtor"])
df_full_reviews["local_id"] = le_local.fit_transform(df_full_reviews["local"])

df_full_reviews[["usuario_id", "produto_id", "organico", "avaliacao", "produtor_id", "local_id", "latitude", "longitude"]].head(20)

# Features usadas para similaridade (excluindo avaliação que será alvo)
feature_cols = ["produto_id", "organico", "produtor_id", "local_id", "latitude", "longitude"]

# Matriz de características e vetor de alvo
X = df_full_reviews[feature_cols].values
y = df_full_reviews['avaliacao'].values

# Modelo de vizinhos mais próximos
knn_model = NearestNeighbors(n_neighbors=6, metric='euclidean')
knn_model.fit(X)

NearestNeighbors(metric='euclidean', n_neighbors=6)

In [10]:
def get_recommendation_candidates(desired_products, producer, location):
    # global df_full_reviews

    if isinstance(desired_products, str):
        desired_products = [desired_products]

    candidates = df_full_reviews[
        (df_full_reviews['produto'].isin(desired_products)) |
        (df_full_reviews['nome_produtor'] == producer) |
        (df_full_reviews['local'] == location)
    ].copy()

    # Remove combinações exatas
    candidates = candidates[~(
        (candidates['produto'].isin(desired_products)) &
        (candidates['nome_produtor'] == producer) |
        (candidates['produto'].isin(desired_products))
    )]

    return candidates


def normalize_distance(candidates, latitude, longitude):
    candidates['distancia_km'] = candidates.apply(
        lambda row: get_distance((latitude, longitude), (row['latitude'], row['longitude'])),
        axis=1
    )
    max_dist = candidates['distancia_km'].max()
    candidates['proximidade'] = 1 - (candidates['distancia_km'] / max_dist)
    return candidates


def calculate_average_rating(candidates):
    candidates['media_produtor_produto'] = (
        candidates.groupby(['produto', 'nome_produtor'])['avaliacao']
        .transform('mean')
    )
    candidates['avaliacao_norm'] = candidates['media_produtor_produto'] / 5.0
    return candidates


def calculate_score(recommendation_type: int, is_organic: int, feature_values: list) -> float:
    # Inicialização dos pesos padrão
    weights = {
        "rating": 0.5,
        "proximity": 0.5,
        "organic": 0.0
    }

    if recommendation_type in (0, 2):
        if is_organic == 1:
            weights.update({
                "rating": 0.3,
                "proximity": 0.5,
                "organic": 0.2
            })
        else:
            weights["organic"] = -1.0

    elif recommendation_type == 1:
        weights.update({
            "rating": 0.7,
            "proximity": 0.3,
            "organic": 0.0
        })
        # Para tipo 1, só rating e proximidade são usados
        return (
            weights["rating"] * feature_values[0]
            + weights["proximity"] * feature_values[1]
        )

    # Cálculo do score final para tipos 0 e 2
    return (
        weights["rating"] * feature_values[0]
        + weights["proximity"] * feature_values[1]
        + weights["organic"] * feature_values[2]
    )

In [11]:
def recommend_best_products(desired_products, producer, location, organic, latitude, longitude):
    candidates = get_recommendation_candidates(desired_products, producer, location)

    if candidates.empty:
        return pd.DataFrame({'mensagem': ['Nenhuma recomendação alternativa encontrada.']})

    candidates = normalize_distance(candidates, latitude, longitude)
    candidates = calculate_average_rating(candidates)

    features_values = [candidates['avaliacao_norm'], candidates['proximidade'], candidates['organico']]
    candidates["score"] = calculate_score(0, organic, features_values)

    top_recommendations = (
        candidates.sort_values(by='score', ascending=False)
        .drop_duplicates(subset=['produto', 'nome_produtor'])
        .head(5)
    )

    return top_recommendations[[
        'produto', 'nome_produtor', 'local', 'organico',
        'media_produtor_produto', 'distancia_km', 'score'
    ]].round(2)

In [12]:
# Recomenda os melhores produtos de determinado produtor com base no produto fornecido e se é organico
recommend_best_products(
    desired_products=['Uva', "Limão"],
    producer='Asphor',
    location='Gama',
    organic=0,
    latitude=-16.0170857,
    longitude=-48.0653054
)

,produto,nome_produtor,local,organico,media_produtor_produto,distancia_km,score
1294,Brócolis,Agrifam,Gama,0,4.50,0.0,0.95
2961,Quiabo,Agrifam,Gama,0,4.10,0.0,0.91
1431,Coco,Coopbrasil,Gama,0,3.83,0.0,0.88
3879,Mandioca,Agrifam,Gama,0,3.71,0.0,0.87
5921,Beterraba,Coopbrasil,Gama,0,3.62,0.0,0.86


In [13]:
def get_producer_recomendation(df_reviews, product):
    candidates = df_reviews[df_reviews['produto'] == product].copy()

    if product.strip() == "":
        candidates = df_reviews.copy()

    if candidates.empty:
        return pd.DataFrame({'mensagem': ['Nenhum produtor encontrado para este produto.']})
    
    # candidates = candidates.sort_values('media_avaliacao', ascending=False)
    return candidates.drop_duplicates(subset=['nome_produtor'], keep='first')

def calculate_average_producer_rating(candidates):
    candidates = (
        candidates.groupby(['nome_produtor', 'local', 'latitude', 'longitude', 'organico'])
        .agg(media_avaliacao=('avaliacao', 'mean'))
        .reset_index()
    )
    candidates['avaliacao_norm'] = candidates['media_avaliacao'] / 5.0
    return candidates

In [14]:
def recommend_best_productors(product, latitude, longitude, top_n=5):
    candidates = get_producer_recomendation(df_full_reviews, product)
    candidates = calculate_average_producer_rating(candidates)
    candidates = normalize_distance(candidates, latitude, longitude)

    features_values = [candidates['avaliacao_norm'], candidates['proximidade'], candidates["organico"]]
    candidates["score"] = calculate_score(1, 0, features_values)

    top_result = candidates.sort_values(by='score', ascending=False).head(top_n)

    return top_result[[
        'nome_produtor', 'local', 'organico',
        'media_avaliacao', 'distancia_km', 'score'
    ]].round(2)

In [15]:
# Recomenda os melhores produtores de determinado produto considerando a localização como foco
recommend_best_productors(
    product='Maracujá',
    latitude=-15.7183687,
    longitude=-47.9950273
)

,nome_produtor,local,organico,media_avaliacao,distancia_km,score
6,Aspronte,Recanto das Emas,1,5.0,21.56,0.81
9,Coopebraz,Recanto das Emas,1,5.0,21.56,0.81
11,Cooperbrasília,Sobradinho,1,5.0,23.77,0.79
3,Aspaf,Guará,1,4.0,11.80,0.76
7,Astraf,Guará,0,4.0,11.80,0.76


In [16]:
def get_products_recomendation(df_full_reviews, producer, unwanted_products):
    if unwanted_products is None:
        unwanted_products = []

    candidates = df_full_reviews[
        (df_full_reviews['nome_produtor'] == producer) &
        (~df_full_reviews['produto'].isin(unwanted_products))
    ].copy()

    if candidates.empty:
        return pd.DataFrame({'mensagem': ['Nenhum produtor encontrado para este produto.']})
    
    return candidates


In [17]:
def recommend_best_product_productors(producer, local, organic, latitude, longitude, unwanted_products=None):

    candidates = get_products_recomendation(df_full_reviews, producer, unwanted_products)
    candidates = normalize_distance(candidates, latitude, longitude)
    candidates = candidates.drop_duplicates(subset=['produto', 'nome_produtor'])
    candidates = calculate_average_rating(candidates)

    features_values = [candidates['avaliacao_norm'], candidates['proximidade'], candidates['organico']]
    candidates["score"] = calculate_score(2, organic, features_values)
    
    resultado = candidates.sort_values(by='score', ascending=False).head(5)

    return resultado[['produto', 'nome_produtor', 'local', 'organico',
                      'media_produtor_produto', 'distancia_km', 'score']]


In [18]:
# Recomenda os melhores produtos de um determinado produtor

recommend_best_product_productors(
    producer='Cooperbrasília',
    local='Gama',
    organic=0,
    latitude=-15.650053,
    longitude=-47.784845,
    unwanted_products=["Mandioca", "Morango"]
)

,produto,nome_produtor,local,organico,media_produtor_produto,distancia_km,score
297,Limão,Cooperbrasília,Sobradinho,0,4.0,0.000035,0.899999
232,Pitaia,Cooperbrasília,Sobradinho,0,3.0,0.000035,0.799999
38,Uva,Cooperbrasília,Sobradinho,0,2.0,0.000035,0.699999
981,Manga,Cooperbrasília,Sobradinho,0,2.0,0.000035,0.699999
629,Abacate,Cooperbrasília,São Sebastião,0,5.0,28.066938,0.500000


In [19]:
full_data = {
    "model": knn_model
}

with open("./data/model/model.pkl", "wb") as file:
    pickle.dump(full_data, file)

In [20]:
# Salvar encoders, modelo e metadados
resources = {
    "knn_model": knn_model,
    "le_usuario": le_usuario,
    "le_produto": le_produto,
    "le_produtor": le_produtor,
    "le_local": le_local,
    "feature_cols": feature_cols,
    "cities_list": cities_list,
    "products_list": products_list,
    "producers_formatted": producers_formatted
}

joblib.dump(resources, './data/model/full_resources.pkl')

# Salvar DataFrames em formato eficiente
df_full_reviews.to_parquet('./data/datasets/df_full_reviews.parquet')
df_products.to_parquet('./data/datasets/producers.parquet')